<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Financial Consolidation Dashboard 

**Tags:** #dashboard #plotly #dash #naas #asset #automation #ai #analytics

**Author:** [Meriem Si](https://www.linkedin.com/in/meriem-si-104236181/)

This notebook enables you to generate a dashboard to follow the financial consolidation.

## Input

### Import libraries

In [29]:
import dash
from dash import html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
from naas_drivers import gsheet
from dash_bootstrap_components._components.Container import Container
from plotly.subplots import make_subplots

### Defining the port of the dashboard

In [ ]:
DASH_PORT = 8050

### Setup Google Sheets

In [31]:
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/127PqDiuz5CXa8tTzhhgUl_ziMYThwZhfWOi86Bo-h-g/edit?usp=sharing"
SHEET_NAME = "KPIS"
SHEET_NAME_2 = "REVENUE"
SHEET_NAME_3 = "OPERATIONAL_EXPENSES"
SHEET_NAME_4 = "EARNING_BEFORE_INTREST_AND_TAXES"

### App design

In [ ]:
APP_TITLE = 'Financial Consolidation'
APP_LOGO = "https://images.plot.ly/logo/new-branding/plotly-logomark.png"

## Model

### Data dropdowns

In [ ]:
entities = [
    "Entity1",
    "Entity2"
]

scenarios = [
    "2022",
    "2021",
    "2020"
]

### Get Kpis data

In [37]:
df_hkpis = gsheet.connect(SPREADSHEET_URL).get(SHEET_NAME)
df_hkpis["VALUE_D"] = df_hkpis["VALUE"].astype(str) + " " + df_hkpis["UNITS"].astype(str)
print("Nb row:", len(df_hkpis))
df_hkpis.head(6)

Nb row: 36


,ENTITY,SCENARIO,LABEL,VALUE,UNITS,VALUE_D
0,Entity1,2022,Revenue target,80,%,80 %
1,Entity1,2022,COGS ratio,20,%,20 %
2,Entity1,2022,Gross Profit Margin,84,%,84 %
3,Entity1,2022,Opex Ratio,74,%,74 %
4,Entity1,2022,Ebit Margin,18,%,18 %
5,Entity1,2022,Net Profit Margin,14,%,14 %


### Data...

In [ ]:
data = [
    {"DATE": "Jan 15", "VALUE": 40, "VARV": 12},
    {"DATE": "Feb 15", "VALUE": 40, "VARV": 25.2},
    {"DATE": "Mar 15", "VALUE": 45, "VARV": 45.42},
    {"DATE": "Apr 15", "VALUE": 43, "VARV": 54.9},
    {"DATE": "May 15", "VALUE": 42, "VARV": 64},
    {"DATE": "Jun 15", "VALUE": 46, "VARV": 16.2},
    {"DATE": "Jul 15", "VALUE": 45, "VARV": 26.3},
    {"DATE": "Aug 15", "VALUE": 41, "VARV": 26.6},
    {"DATE": "Sep 15", "VALUE": 42, "VARV": 46.8},
    {"DATE": "Oct 15", "VALUE": 43, "VARV": 37},
    {"DATE": "Nov 15", "VALUE": 44, "VARV": 27.2},
    {"DATE": "Dec 15", "VALUE": 45, "VARV": 57.5},
]

In [ ]:
data2 = [
    {"DATE": "Jan 15", "VALUE": 'General', "COUNT": 10},
    {"DATE": "Jan 15", "VALUE": 'Marketing', "COUNT": 32},
    {"DATE": "Jan 15", "VALUE": 'Sales', "COUNT": 20},
    {"DATE": "Jan 15", "VALUE": 'IT', "COUNT": 55},
    {"DATE": "Feb 15", "VALUE": 'General', "COUNT": 25},
    {"DATE": "Feb 15", "VALUE": 'Marketing', "COUNT": 65},
    {"DATE": "Feb 15", "VALUE": 'Sales', "COUNT": 15},
    {"DATE": "Feb 15", "VALUE": 'IT', "COUNT": 5},
    {"DATE": "Mar 15", "VALUE": 'General', "COUNT": 35},
    {"DATE": "Mar 15", "VALUE": 'Marketing', "COUNT": 45},
    {"DATE": "Mar 15", "VALUE": 'Sales', "COUNT": 25},
    {"DATE": "Mar 15", "VALUE": 'IT', "COUNT": 5},
    {"DATE": "Apr 15", "VALUE": 'General', "COUNT": 25},
    {"DATE": "Apr 15", "VALUE": 'Marketing', "COUNT": 5},
    {"DATE": "Apr 15", "VALUE": 'Sales', "COUNT": 15},
    {"DATE": "Apr 15", "VALUE": 'IT', "COUNT": 10},
    {"DATE": "May 15", "VALUE": 'General', "COUNT": 12},
    {"DATE": "May 15", "VALUE": 'Marketing', "COUNT": 14},
    {"DATE": "May 15", "VALUE": 'Sales', "COUNT": 20},
    {"DATE": "May 15", "VALUE": 'IT', "COUNT": 19},
    {"DATE": "Jun 15", "VALUE": 'General', "COUNT": 16},
    {"DATE": "Jun 15", "VALUE": 'Marketing', "COUNT": 42},
    {"DATE": "Jun 15", "VALUE": 'Sales', "COUNT": 10},
    {"DATE": "Jun 15", "VALUE": 'IT', "COUNT": 5},
    {"DATE": "Jul 15", "VALUE": 'General', "COUNT": 22},
    {"DATE": "Jul 15", "VALUE": 'Marketing', "COUNT": 26},
    {"DATE": "Jul 15", "VALUE": 'Sales', "COUNT": 15},
    {"DATE": "Jul 15", "VALUE": 'IT', "COUNT": 5},
    {"DATE": "Aug 15", "VALUE": 'General', "COUNT": 12},
    {"DATE": "Aug 15", "VALUE": 'Marketing', "COUNT": 17},
    {"DATE": "Aug 15", "VALUE": 'Sales', "COUNT": 20},
    {"DATE": "Aug 15", "VALUE": 'IT', "COUNT": 25},
    {"DATE": "Sep 15", "VALUE": 'General', "COUNT": 25},
    {"DATE": "Sep 15", "VALUE": 'Marketing', "COUNT": 17},
    {"DATE": "Sep 15", "VALUE": 'Sales', "COUNT": 18},
    {"DATE": "Sep 15", "VALUE": 'IT', "COUNT": 19},
    {"DATE": "Oct 15", "VALUE": 'General', "COUNT": 23},
    {"DATE": "Oct 15", "VALUE": 'Marketing', "COUNT": 22},
    {"DATE": "Oct 15", "VALUE": 'Sales', "COUNT": 26},
    {"DATE": "Oct 15", "VALUE": 'IT', "COUNT": 8},
    {"DATE": "Nov 15", "VALUE": 'General', "COUNT": 4},
    {"DATE": "Nov 15", "VALUE": 'Marketing', "COUNT": 41},
    {"DATE": "Nov 15", "VALUE": 'Sales', "COUNT": 3},
    {"DATE": "Nov 15", "VALUE": 'IT', "COUNT": 6},
    {"DATE": "Dec 15", "VALUE": 'General', "COUNT": 4},
    {"DATE": "Dec 15", "VALUE": 'Marketing', "COUNT": 20},
    {"DATE": "Dec 15", "VALUE": 'Sales', "COUNT": 31},
    {"DATE": "Dec 15", "VALUE": 'IT', "COUNT": 16},
]

In [ ]:
data3 = [
    {"DATE": "Jan 15", "VALUE1": 40, "VALUE2": 50},
    {"DATE": "Feb 15", "VALUE1": 60, "VALUE2": 52},
    {"DATE": "Mar 15", "VALUE1": 45, "VALUE2": 54},
    {"DATE": "Apr 15", "VALUE1": 49, "VALUE2": 56},
    {"DATE": "May 15", "VALUE1": 55, "VALUE2": 58},
    {"DATE": "Jun 15", "VALUE1": 66, "VALUE2": 60},
    {"DATE": "Jul 15", "VALUE1": 45, "VALUE2": 62},
    {"DATE": "Aug 15", "VALUE1": 61, "VALUE2": 64},
    {"DATE": "Sep 15", "VALUE1": 72, "VALUE2": 66},
    {"DATE": "Oct 15", "VALUE1": 83, "VALUE2": 68},
    {"DATE": "Nov 15", "VALUE1": 64, "VALUE2": 70},
    {"DATE": "Dec 15", "VALUE1": 85, "VALUE2": 72},
]

#### Highlighted KPIs

In [ ]:
# Dataframe is returned
df_hkpis = naas_drivers.gsheet.connect(spreadsheet_id).get(
    sheet_name="KPIS"
)

#### Gross Profit Margin

In [ ]:
def gross_profit_margin():

    return fig  


#### Opex Ratio

In [ ]:
def opex_ratio():

    return fig  


#### Ebit Margin

In [ ]:
def ebit_margin():

    return fig  


#### Net Profit Margin

In [ ]:
def net_profit_margin():

    return fig  


#### Income Statement

In [ ]:
def income_statement():

    return fig  


#### Revenue

In [ ]:
df = pd.DataFrame(data)
df

def create_barlinechart(df,
                        label="DATE",
                        value="VARV",
                        varv="VALUE",
                        xaxis_title=None,
                        yaxis_title_r=None,
                        yaxis_title_l=None):    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(
            x=df[label],
            y=df[value],
            name="Growth",
            marker=dict(color="#ADD8E6"),
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            x=df[label],
            y=[86,87,98,89,90,91,92,113,94,95,96,117],
            mode="lines",
            name="Revenue",
            line=dict(color="royalblue", width=2.5),
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            x=df[label],
            y=[78,69,80,81,82,93,84,85,96,87,88,119],
            name="Target",
            line=dict(color='green', width=4, dash='dot'),
        ),
        secondary_y=False,
    )

    # Add figure title
    fig.update_layout(
        title= "Revenue",
        #title_font=dict(family="Arial", size=18, color="black"),
        #legend=None,
        title_x=0.1,
        plot_bgcolor="white",
        height=300,
        paper_bgcolor="white",
        xaxis_title=xaxis_title,
        xaxis_title_font=dict(family="Arial", size=10, color="black"),
    )

    # Set y-axes titles
    fig.update_yaxes(
        title_text=yaxis_title_r,
        title_font=dict(family="Arial", size=10, color="black"),
        secondary_y=False
    )
    fig.update_yaxes(
        title_text=yaxis_title_l,
        title_font=dict(family="Arial", size=10, color="black"),
        secondary_y=True
    )
    fig.update_traces(showlegend=False)
    fig.show()
    return fig

fig = create_barlinechart(df,
                          yaxis_title_r=None,
                          yaxis_title_l=None)

### Operational Expenses

In [ ]:
df2 = pd.DataFrame(data2)
df2

In [ ]:
def stacked_bar(df2, labels, values):
    fig3 = px.bar(df2, x="DATE", y="COUNT", color="VALUE",
                 width=500, height=400,
                 color_discrete_map={
                     'General': '#5ac73c',
                     'Marketing': '#6dabc7',
                     'Sales':'#307a37',
                     'IT':'#30337a'
                 })
    fig3.update_traces(width=0.5)
    fig3.update_layout(yaxis_tickformat = ',',
                       title_text='Operational Expenses',
                       title_x=0.1,
                       plot_bgcolor="#ffffff",)    
    fig3.show()
    return fig3
stacked_bar = stacked_bar(df2, None, None)

### Earnings before Intrest and Taxes

In [ ]:
df3 = pd.DataFrame(data3)
df3

In [ ]:
def create_doublelinechart(df3,
                        label="DATE",
                        value1="VALUE1",
                        value2="VALUE2",
                        xaxis_title="Months"):    
    fig2 = go.Figure()
    # Add traces

    fig2.add_trace(
        go.Scatter(
            x=df3[label],
            y=df3[value1],
            mode="lines",
            name="EBIT Actual",
            line=dict(color="#0A66C2", width=2.5),
        ),
    )
    
    fig2.add_trace(
        go.Scatter(
            x=df3[label],
            y=df3[value2],
            mode="lines",
            name="EBIT Target",
            line=dict(color='green', width=4, dash='dot'),
        ),
    )  
    # Add figure title
    fig2.update_layout(
        yaxis_tickformat = ',',
        title_text="Earnings before Intrest and Taxes",
        title_x=0.1,
        #title_font=dict(family="Arial", size=18, color="black"),
        #legend=None,
        plot_bgcolor="#ffffff",
        width=500,
        height=400,
        #paper_bgcolor="white",
        #xaxis_title=xaxis_title,
        #xaxis_title_font=dict(family="Arial", size=10, color="black"),
    )

    fig2.show()
    return fig2

doubleline = create_doublelinechart(df3)

### Design

#### Create Dropdown

In [ ]:
# Entity's dropdown list
dropdown_entity = dcc.Dropdown(
    id='entity',
    options=[{'label': i, 'value': i} for i in entities],
    placeholder='Entity',
    value=entities[0],
)

# Scenario's dropdown list
dropdown_scenario = dcc.Dropdown(
    id='scenario',
    options=[{'label': i, 'value': i} for i in scenarios],
    placeholder='Scenario',
    value=scenarios[0],
)

#### Create Navbar

In [ ]:
navbar = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                # Use row and col to control vertical alignment of logo / brand
                dbc.Row(
                    [
                        dbc.Col(html.Img(src=APP_LOGO, height="20")),
                        dbc.Col(dbc.NavbarBrand(APP_TITLE, className="ms-2")),
                    ],
                    align="center",
                    className="g-0",
                ),
            ),
            dbc.NavbarToggler(id="navbar-toggler", n_clicks=0),
            dbc.Collapse(
                dbc.Nav(
                    [
                        html.Div(
                            [
                                html.Div(className="w-100"),
                                html.Div(className="w-100"),
                                html.Div(dropdown_entity, className="w-100"),
                                html.Div(dropdown_scenario, className="w-100")
                            ],
                            className="pt-1 pb-1 d-grid gap-2 d-md-flex w-100")

                    ],
                    className="ms-auto w-100",
                    navbar=True,
                ),
                id="navbar-collapse",
                navbar=True,
                is_open=False,
            ),
        ],
    ),
    color="#163b78",
    dark=True,
)

#### Create app layout

In [ ]:
app = dash.Dash(requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/', 
                external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{'name': 'viewport','content': 'width=device-width, initial-scale=1.0'}]
                )   
#app = dash.Dash() if you are not in Naas

app.layout = html.Div(
[
    # Navbar
    navbar,

    #HKPIS
    html.Div([
        dbc.Row([

            dbc.Col(
                html.Div(
                    dbc.Card(
                        dbc.CardBody([
                            html.H5("Revenue target", className="card-title",style={'text-align':'center'}),
                            html.P(
                                "80%",
                                className="card-text",
                                style={'text-align':'center'},
                            ),
                        ],),color="#163b78", inverse=True,
                    ),
                ),className="gx-5 g-2",xs=12,sm=12,md=12,lg=2,xl=2,
            ),
            dbc.Col(
                html.Div(
                    dbc.Card(
                        dbc.CardBody([
                            html.H5("COGS ratio", className="card-title",style={'text-align':'center'}),
                            html.P(
                                "20%",
                                className="card-text",
                                style={'text-align':'center'},
                            ),
                        ],),color="#4c718f", inverse=True,
                    ),
                ),className="gx-5 g-2",xs=12,sm=12,md=12,lg=2,xl=2,
            ),

            dbc.Col(
                html.Div(
                    dbc.Card(
                        dbc.CardBody([
                            html.H5("Gross Profit Margin", className="card-title",style={'text-align':'center'}),
                            html.P(
                                "84%",
                                className="card-text",
                                style={'text-align':'center'},
                            ),
                        ],),color="#62a0a6", inverse=True,
                    ),
                ),className="gx-5 g-2",xs=12,sm=12,md=12,lg=2,xl=2,
            ),

            dbc.Col(
                html.Div(
                    dbc.Card(
                        dbc.CardBody([
                            html.H5("Opex Ratio", className="card-title",style={'text-align':'center'}),
                            html.P(
                                "74%",
                                className="card-text",
                                style={'text-align':'center'},
                            ),
                        ],),color="#64abbd", inverse=True,
                    ),
                ),className="gx-5 g-2",xs=12,sm=12,md=12,lg=2,xl=2,
            ),

            dbc.Col(
                html.Div(
                    dbc.Card(
                        dbc.CardBody([
                            html.H5("Ebit Margin", className="card-title", style={'text-align':'center'}),
                            html.P(
                                "18%",
                                className="card-text",
                                style={'text-align':'center'},
                            ),
                        ],),color="#45a127", inverse=True,
                    ),
                ),className="gx-5 g-2",xs=12,sm=12,md=12,lg=2,xl=2,
            ),

            dbc.Col(
                html.Div(
                    dbc.Card(
                        dbc.CardBody([
                            html.H5("Net Profit Margin", className="card-title",style={'text-align':'center'}),
                            html.P(
                                "14%",
                                className="card-text",
                                style={'text-align':'center'},
                            ),
                        ],),color="success", inverse=True,
                    ),
                ),className="gx-5 g-2",xs=12,sm=12,md=12,lg=2,xl=2,
            ),
            ],),


        #SPACE
        html.Br(),

        #CHARTS
        dbc.Row([
            dbc.Col([
                html.Div(
                    dcc.Graph(id="test2",figure=fig)
                ),
            dbc.Row([
                dbc.Col(
                    html.Div(
                        dcc.Graph(id="test4",figure=stacked_bar)
                    ), xs=12,sm=12,md=12,lg=5,xl=5,
                ),
                dbc.Col(
                    html.Div(
                        dcc.Graph(id="test5",figure=doubleline)
                    ),xs=12,sm=12,md=12,lg=4,xl=4,
                ),

            ],)

            ],

            ),
            dbc.Col(
                html.Div(
                    dbc.Card(
                        dbc.CardBody([
                            html.H5("Income Statement", className="card-title"),


                            dbc.Row([
                                dbc.Col([
                                    html.P("Revenue",className="card-text",style={"font-weight": "bold"})
                                ]),
                                dbc.Col([
                                    html.P("1,305,507 €",style={'text-align':'right'})
                                ]),
                             ]),

                            dbc.Row([
                                dbc.Col([
                                    html.P("COGS",className="card-text",style={"font-weight": "bold"})
                                ], width=4),
                                dbc.Col([
                                    html.P("208,453 €",style={'text-align':'right'})
                                ]),
                             ]),

                            html.Hr(style={'borderWidth': "0.5vh", "width": "100%", "color": "blak"}),

                            dbc.Row([
                                dbc.Col([
                                    html.P("GROSS PROFIT",className="card-text",style={"font-weight": "bold"})
                                ]),
                                dbc.Col([
                                    html.P("1,097,054 €",style={'text-align':'right'})
                                ]),
                             ]),                                


                            dbc.Row([
                                dbc.Col([
                                    html.P("OPEX",className="card-text",style={"font-weight": "bold"})
                                ]),
                                dbc.Col([
                                    html.P("815,306 €",style={'text-align':'right'})
                                ]),
                             ]),

                            dbc.Row([
                                dbc.Col([
                                    html.P("Sales",className="card-text", style={'margin-left':25})
                                ]),
                                dbc.Col([
                                    html.P("279,886 €",style={'text-align':'right'})
                                ]),
                             ]),

                            dbc.Row([
                                dbc.Col([
                                    html.P("Marketing",className="card-text", style={'margin-left':25})
                                ]),
                                dbc.Col([
                                    html.P("192,710 €",style={'text-align':'right'})
                                ]),
                             ]),

                            dbc.Row([
                                dbc.Col([
                                    html.P("IT",className="card-text", style={'margin-left':25})
                                ]),
                                dbc.Col([
                                    html.P("192,656 €",style={'text-align':'right'})
                                ]),
                             ]),

                            dbc.Row([
                                dbc.Col([
                                    html.P("General & Admin ",className="card-text", style={'margin-left':25})
                                ]),
                                dbc.Col([
                                    html.P("150,054 €",style={'text-align':'right'})
                                ]),
                             ]),

                            dbc.Row([
                                dbc.Col([
                                    html.P("OTHER INCOME",className="card-text",style={"font-weight": "bold"})
                                ]),
                                dbc.Col([
                                    html.P("2,130 €",style={'text-align':'right'})
                                ]),
                             ]),

                            dbc.Row([
                                dbc.Col([
                                    html.P("OTHER EXPENSES",className="card-text",style={"font-weight": "bold"})
                                ]),
                                dbc.Col([
                                    html.P("51,195 €",style={'text-align':'right'})
                                ]),
                             ]),

                            html.Hr(style={'borderWidth': "0.5vh", "width": "100%", "color": "blak"}),

                            dbc.Row([
                                dbc.Col([
                                    html.P("EBIT",className="card-text",style={"font-weight": "bold"})
                                ]),
                                dbc.Col([
                                    html.P("232,684 €",style={'text-align':'right'})
                                ]),
                             ]),

                            dbc.Row([
                                dbc.Col([
                                    html.P("Intrest and Tax",className="card-text", style={'margin-left':25})
                                ]),
                                dbc.Col([
                                    html.P("1,305,507 €",style={'text-align':'right'})
                                ]),
                             ]),

                            html.Hr(style={'borderWidth': "0.5vh", "width": "100%", "color": "blak"}),

                            dbc.Row([
                                dbc.Col([
                                    html.P("NET PROFIT",className="card-text",style={"font-weight": "bold"})
                                ]),
                                dbc.Col([
                                    html.P("194,440 €",style={'text-align':'right'})
                                ]),
                             ]),                  
                        ],),
                    ),
                ),className="gx-5 g-2",xs=12,sm=12,md=12,lg=4,xl=4,
            ),

        ],),



    ],),
],
)
    
# add callback for toggling the collapse on small screens
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
    [State("navbar-collapse2", "is_open")],   
)
@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

## Output

### Generate URL and show logs

In [27]:
if __name__ == '__main__':
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")